# Evaluation

In [2]:
import pandas as pd
from datetime import timedelta 
import numpy as np
import config
import warnings
warnings.filterwarnings('ignore')

## Entire main function

In [4]:
import pandas as pd
from datetime import timedelta 
import numpy as np
import config
from sklearn import ensemble
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [5]:
def patient_cutoff(df, cutoff_year, cutoff_visits):
    # patients must have this many years of data to be included.
    frames = []
    id_list = df.eye_id.unique()
    for eye in id_list:
        pdf = df[df.eye_id == eye]
        dates = (pd.to_datetime(pdf.admission_date)).to_list()
        if ((dates[-1] - dates[0]).days)/365 >= cutoff_year and len(pdf)>=cutoff_visits: 
            frames.append(pdf)
    return pd.concat(frames)

def cut_time(df, cutoff_time):
        # shortens a patient's dataframe to x years after initiation.
        frames = []
        id_list = df.eye_id.unique()
        for eye in id_list:
            pdf = df[df.eye_id == eye]
            pdf.admission_date = pd.to_datetime(pdf.admission_date)
            dates = pdf['admission_date'].to_list()
            first = pd.to_datetime(dates[0])
            cutoff = first + timedelta(days=cutoff_time*365)
            pdf = pdf[pdf['admission_date'] <= cutoff]
            if len(pdf) > 0: frames.append(pdf)
        return pd.concat(frames)
    
def impute_pdf(df):
    fill_NaN = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    imputed_df = pd.DataFrame(fill_NaN.fit_transform(df))
    imputed_df.columns = df.columns
    imputed_df.index = df.index
    imputed_df.fillna(0, inplace=True)
    return imputed_df

def column_names(i):
    return [f'va_{i}', f'int_{i}']

def column_builder(i):
    lst = []
    for visits in range(1, i+1):
        lst.extend(column_names(visits))
    lst.append('mean_vision'), lst.append('std_vision')
    lst.append('target_va')
    lst.remove('int_1')
    return lst

def reshape_pdf(pdf, n_visits):
    nums, columns = [], column_builder(n_visits)
    pdf.fillna(0, inplace=True)
    for i in range(n_visits): 
        nums.append(pdf.visual_acuity.iloc[i])
        if i != 0: nums.append((pdf.admission_date.iloc[i] - pdf.admission_date.iloc[i-1]).days)
    if n_visits > 6: nums.append(np.mean(pdf.visual_acuity))
    else: nums.append(np.mean(pdf.visual_acuity.iloc[:n_visits+1]))
    if n_visits > 3: nums.append(np.std(pdf.visual_acuity))
    else: nums.append(np.std(pdf.visual_acuity.iloc[:n_visits+1]))
    nums.append(pdf.visual_acuity.iloc[-1])
    return pd.DataFrame(data=[nums], columns=columns)

def encode_gender(g):
    return 0 if g == "Male" else 1

def reshape_df(df, n_visits):
    eyes = df.eye_id.unique()
    frames = []
    for eye in eyes:
        pdf = df[df.eye_id == eye]
        try: frames.append(reshape_pdf(pdf, n_visits))
        except: pass
    return pd.concat(frames)

def save_df_patients(n_years, n_visits=4):
    df = pd.read_csv("~/Documents/github/paper/input/raw_test_data_cleaned.csv")
    df.drop(columns=['actual_drug_today', 'next_interval_in_weeks', 'InjNext',
                     'laterality'], inplace=True)
    df["irf"] = 0
    df["srf"] = 0
    df = patient_cutoff(df, n_years, 4)
    df = cut_time(df, n_years)
    df = reshape_df(df, n_visits)
    return df

In [6]:
def get_train_test(n_years):
    test_df = save_df_patients(n_years)
    test_df.reset_index(drop=True, inplace=True)

    train_df = pd.read_csv(config.TRAINING_FILE[n_years-1])
    features = ['va_1', 'va_2', 'int_2', 'va_3', 'int_3', 'va_4', 'int_4', 
                'mean_vision', 'std_vision', 'target_va']
    train_df = train_df[features]
    return train_df, test_df

tabnet_params = {"optimizer_fn":torch.optim.Adam,
                 "verbose":0,
                 "optimizer_params":dict(lr=2e-2),
                 "scheduler_params":{"step_size":50, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

def score(model, X, y, cv=5, scoring='neg_mean_squared_error'):
    scores = cross_val_score(model, X, y, cv=cv, scoring=scoring)
    return np.mean(scores), np.std(scores)

def run(df, clf, rf=False):
    # create inputs and targets
    X, y = df.drop(columns=['target_va']).values, df.target_va.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
    # score the model
    if rf: clf.fit(X_train, y_train)
    else: kfold_tabnet(clf, X, y.reshape(-1, 1))

def kfold_tabnet(clf, X, y):
    rmses = []
    for i in range(5):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        rmse = fit_tabnet(clf, X_train, y_train, X_test, y_test)
        rmses.append(rmse)
    final_rmse, rmse_std = np.round(np.mean(rmses), 2), np.round(np.std(rmses), 2)
    print(f"Train-set RMSE: mean={final_rmse}, std={rmse_std}")
    
def fit_tabnet(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test)],
            eval_metric=['rmse'], patience=1000, max_epochs=10000)
    preds = clf.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse

def evaluate(test_df, clf):
    # create inputs and targets
    X, y = test_df.drop(columns=['target_va']).values, test_df.target_va.values
    # find rmse 
    preds = clf.predict(X)
    rmse = np.sqrt(mean_squared_error(y, preds))
    print(f"The RMSE on the test-set was {rmse} logMAR letters.")

In [7]:
def main(n_years, rf=False):
    train_df, test_df = get_train_test(n_years)
    if rf:
        clf = ensemble.RandomForestRegressor()
    else:
        clf = TabNetRegressor(**tabnet_params)
    run(train_df, clf, rf)
    evaluate(test_df, clf)

In [41]:
main(3, rf=True)

The RMSE on the test-set was 15.321285476724013 logMAR letters.


## Vanilla NN

In [88]:
n_years=3
train_df = pd.read_csv(config.TRAINING_FILE[n_years-1])

In [96]:
X, y = train_df.drop(columns=['target_va']).values, train_df.target_va.values
X = torch.from_numpy(X)
y = torch.from_numpy(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [97]:
train_data, test_data = [], []
for i in range(len(X_train)):
    train_data.append([X_train[i], 
                       y_train[i]])
for i in range(len(X_test)):
    test_data.append([X_test[i], 
                       y_test[i]])

In [98]:
train_loader = torch.utils.data.DataLoader(train_data, shuffle=False, 
                                          batch_size=100)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=False,
                                          batch_size=100)

In [99]:
# check shape
i1, l1 = next(iter(train_loader))
print(i1.shape)

torch.Size([100, 17])


In [100]:
import torch.nn.functional as F
import torch.nn as nn

class SimpleNet(nn.Module):
    
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(17, 32)
        self.fc2 = nn.Linear(32, 16)
        self.out = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.25)
        self.double()
        
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.dropout(x)
        x = torch.tanh(self.fc2(x))
        x = x.view(x.size(0), -1)      
        out = self.out(x)
        return out

In [101]:
# check the model works on a random row
model = SimpleNet()
test_row, test_label = X_train[0], y_train[0]
model(test_row.unsqueeze(0))

tensor([[-1.3821]], dtype=torch.float64, grad_fn=<AddmmBackward>)

In [102]:
import datetime
from torch.autograd import Variable

def training_loop(model, optimiser, loss_fn, n_epochs, train_loader):
    model.train()
    for n in range(1, n_epochs+1):
        loss_train = 0.0
        for i, (eyes, labels) in enumerate(train_loader):
            b_x = Variable(eyes)
            b_y = Variable(labels)
            output = model(b_x)
            loss = loss_fn(output, b_y)
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            loss_train += loss.item()
            preds = model(eyes).detach().numpy()
            rmse_train = np.sqrt(mean_squared_error(preds, labels))
            
        # validation metrics
        model.eval()
        with torch.no_grad():
            for eyes, labels in test_loader:
                preds = model(eyes)
                rmse = np.sqrt(mean_squared_error(preds, labels))
            
        if n == 1 or n % 1000 == 0:
            print(f'Epoch [{n}/{n_epochs}], Train Loss: {np.round(loss.item(), 2)}, Train RMSE: {np.round(rmse_train, 2)}, Validation RMSE = {rmse}')
            
            
            

In [103]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

model = SimpleNet()
model.apply(init_weights)
loss_fn = nn.MSELoss()
optimiser = torch.optim.Adam(model.parameters(), lr=1e-2)

training_loop(model,
              optimiser,
              loss_fn, 
              10000,
              train_loader)

Epoch [1/10000], Train Loss: 4710.6, Train RMSE: 68.04, Validation RMSE = 67.30324348370155
Epoch [1000/10000], Train Loss: 386.93, Train RMSE: 19.62, Validation RMSE = 20.708341403008724
Epoch [2000/10000], Train Loss: 386.96, Train RMSE: 19.67, Validation RMSE = 20.70554283423945
Epoch [3000/10000], Train Loss: 387.03, Train RMSE: 19.64, Validation RMSE = 20.817089080849165
Epoch [4000/10000], Train Loss: 386.92, Train RMSE: 19.71, Validation RMSE = 20.93315406531525
Epoch [5000/10000], Train Loss: 386.93, Train RMSE: 19.71, Validation RMSE = 20.939782658761434
Epoch [6000/10000], Train Loss: 386.94, Train RMSE: 19.65, Validation RMSE = 20.935593397850734
Epoch [7000/10000], Train Loss: 386.92, Train RMSE: 19.64, Validation RMSE = 20.91827879282899
Epoch [8000/10000], Train Loss: 386.92, Train RMSE: 19.67, Validation RMSE = 20.94157267515608
Epoch [9000/10000], Train Loss: 386.88, Train RMSE: 19.67, Validation RMSE = 20.965410727044752
Epoch [10000/10000], Train Loss: 386.88, Train R